In [25]:
%pip install langchain-chroma chromadb langchain-ollama --upgrade --quiet


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [9]:
# add this import for running in jupyter notebook
import nest_asyncio

nest_asyncio.apply()

In [24]:
from langchain_chroma import Chroma

from langchain_community.document_loaders import TextLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain.chains import VectorDBQA

In [38]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

template = """Question: {question}

Answer: Let's think step by step."""

prompt = ChatPromptTemplate.from_template(template)

model = OllamaLLM(model="gemma2")

chain = prompt | model

chain.invoke({"question": "What is LangChain?"})

"You're asking about LangChain! It's a pretty cool tool. Here's a breakdown: \n\n**LangChain** is like a toolbox for building applications with language models. Think of it this way: language models (like me!) are great at understanding and generating text, but they need some help to do more complex tasks. \n\nLangChain provides the pieces you need to connect language models to other tools and data sources, allowing you to create powerful applications.  For example, you could use LangChain to:\n\n* **Build chatbots:** Create conversational agents that can answer questions, provide information, and even complete tasks.\n* **Summarize documents:** Automatically generate concise summaries of large chunks of text.\n* **Translate languages:**  Quickly and accurately translate text between different languages.\n* **Write different kinds of creative content:**\n\nGenerate stories, poems, articles, and more.\n\nLangChain is still under development, but it's already being used by developers to 

In [11]:
documents = TextLoader("alice.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

model_name = "sentence-transformers/all-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

Created a chunk of size 1003, which is longer than the specified 1000
Created a chunk of size 1266, which is longer than the specified 1000
/Users/alibayram/Library/Python/3.9/lib/python/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/alibayram/Library/Python/3.9/lib/python/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [18]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

vectordb = Chroma.from_documents(documents=docs, embedding=embeddings, persist_directory=persist_directory)

In [39]:
query = "Bunda PEK fazla şaşılacak bir şey yoktu."

# Query the database
results = vectordb.similarity_search(query, k=2)
results

[Document(metadata={'source': 'alice.txt'}, page_content='Birinci tanık, Şapkacıydı. Buraya bir elinde çay fincanı, diğerinde bir yağlı ekmekle gelmişti. "Böyle geldiğim için, özür dilerim, Majesteleri," diye başladı söze, "fakat buraya çağırıldığımda henüz çayımı bitirmemiştim."\n\n"Bitirmiş olmalıydın," dedi Kral. "Ne zaman başlamıştın?"\n\nŞapkacı, Mart Tavşanına baktı, mahkemeye gelirken Tarla Faresiyle kol kola peşinden gelmişti". "Martın on dördüydü sanırım," dedi.\n\n"On beşi" dedi Mart Tavşanı.\n\n"On altısı" diye ekledi Tarla Faresi.\n\nKral jüriye, "Bunu yazın," dedi ve jüri bu üç tarihi özenle yazı tahtalarına geçirdiler, sonra bunları topladılar ve buldukları yanıtı şilin ve peniye çevirdiler.\n\n\nKral, "Şapkanı çıkar," dedi Şapkacıya.\n\n"Şapka bana ait değil," dedi Şapkacı.\n\n"Çalınmış!" diye vurguladı Kral, ve hemen o anda durumu değerlendirmeye başlayan jüriye döndü.\n\n"Ben bunları satmak için taşıyorum," diye açıklamasını yaptı Şapkacı; "Bunların hiç biri bana ait d

In [41]:
from langchain.chains import RetrievalQA

qachain = RetrievalQA.from_chain_type(model, retriever=vectordb.as_retriever())
res = qachain.invoke({"query": "kim 'Böyle geldiğim için, özür dilerim, Majesteleri.' dedi? Lütfen türkçe cevap veriniz."})
res

{'query': "kim 'Böyle geldiğim için, özür dilerim, Majesteleri.' dedi? Lütfen türkçe cevap veriniz.",
 'result': 'Şapkacı bu cümleyi söyledi. \n\n\n'}